# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), November 20, 2020**

In [1]:
using DataFrames

## Split-apply-combine

### Grouping a data frame

In [2]:
x = DataFrame(id=[1,2,3,4,1,2,3,4], id2=[1,2,1,2,1,2,1,2], v=rand(8))

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,2,2,0.946112
3,3,1,0.759872
4,4,2,0.292012
5,1,1,0.677472
6,2,2,0.802295
7,3,1,0.906859
8,4,2,0.474816


In [3]:
groupby(x, :id)

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,1,1,0.677472
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.292012
2,4,2,0.474816


In [4]:
groupby(x, [])

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,2,2,0.946112
3,3,1,0.759872
4,4,2,0.292012
5,1,1,0.677472
6,2,2,0.802295
7,3,1,0.906859
8,4,2,0.474816


In [5]:
gx2 = groupby(x, [:id, :id2])

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,1,1,0.677472
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.292012
2,4,2,0.474816


In [6]:
parent(gx2) # get the parent DataFrame 

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,2,2,0.946112
3,3,1,0.759872
4,4,2,0.292012
5,1,1,0.677472
6,2,2,0.802295
7,3,1,0.906859
8,4,2,0.474816


In [7]:
vcat(gx2...) # back to the DataFrame, but in a different order of rows than the original

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,1,1,0.677472
3,2,2,0.946112
4,2,2,0.802295
5,3,1,0.759872
6,3,1,0.906859
7,4,2,0.292012
8,4,2,0.474816


In [8]:
DataFrame(gx2) # the same

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,1,1,0.677472
3,2,2,0.946112
4,2,2,0.802295
5,3,1,0.759872
6,3,1,0.906859
7,4,2,0.292012
8,4,2,0.474816


In [9]:
DataFrame(gx2, keepkeys=false) # drop grouping columns when creating a data frame

,v
,Float64
1,0.998123
2,0.677472
3,0.946112
4,0.802295
5,0.759872
6,0.906859
7,0.292012
8,0.474816


In [10]:
groupcols(gx2) # vector of names of grouping variables

2-element Array{Symbol,1}:
 :id
 :id2

In [11]:
valuecols(gx2) # and non-grouping variables

1-element Array{Symbol,1}:
 :v

In [12]:
groupindices(gx2) # group indices in parent(gx2)

8-element Array{Union{Missing, Int64},1}:
 1
 2
 3
 4
 1
 2
 3
 4

In [13]:
kgx2 = keys(gx2)

4-element DataFrames.GroupKeys{GroupedDataFrame{DataFrame}}:
 GroupKey: (id = 1, id2 = 1)
 GroupKey: (id = 2, id2 = 2)
 GroupKey: (id = 3, id2 = 1)
 GroupKey: (id = 4, id2 = 2)

You can index into a `GroupedDataFrame` like to a vector or to a dictionary.
The second form acceps `GroupKey`, `NamedTuple` or a `Tuple`

In [14]:
gx2

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,1,1,0.677472
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.292012
2,4,2,0.474816


In [15]:
k = keys(gx2)[1]

GroupKey: (id = 1, id2 = 1)

In [16]:
ntk = NamedTuple(k)

(id = 1, id2 = 1)

In [17]:
tk = Tuple(k)

(1, 1)

the operations below produce the same result and are fast

In [18]:
gx2[1]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,1,1,0.677472


In [19]:
gx2[k]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,1,1,0.677472


In [20]:
gx2[ntk]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,1,1,0.677472


In [21]:
gx2[tk]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.998123
2,1,1,0.677472


handling missing values

In [22]:
x = DataFrame(id = [missing, 5, 1, 3, missing], x = 1:5)

,id,x
,Int64?,Int64
1,missing,1
2,5,2
3,1,3
4,3,4
5,missing,5


In [23]:
groupby(x, :id) # by default groups include mising values and are not sorted

,id,x
,Int64?,Int64
1,missing,1
2,missing,5
,id,x
,Int64?,Int64
1,3,4


In [24]:
groupby(x, :id, sort=true, skipmissing=true) # but we can change it

,id,x
,Int64?,Int64
1,1,3
,id,x
,Int64?,Int64
1,5,2


### Performing transformations by group using `combine`, `select`, `select!`, `transform`, and `transform!`

In [25]:
using Statistics
using Pipe

In [26]:
ENV["LINES"] = 15 # reduce the number of rows in the output

15

In [27]:
x = DataFrame(id=rand('a':'d', 100), v=rand(100))

,id,v
,Char,Float64
1,b,0.494644
2,a,0.720003
3,a,0.221093
4,a,0.27621
5,d,0.205227
6,c,0.624687
7,c,0.397678
8,b,0.464126
9,a,0.993339


In [28]:
# apply a function to each group of a data frame
# combine keeps as many rows as are returned from the function
@pipe x |> groupby(_, :id) |> combine(_, :v=>mean)

,id,v_mean
,Char,Float64
1,b,0.489617
2,a,0.540168
3,d,0.529731
4,c,0.50889


In [29]:
x.id2 = axes(x, 1)

Base.OneTo(100)

In [30]:
# select and transform keep as many rows as are in the source data frame and in correct order
# additionally transform keeps all columns from the source
@pipe x |> groupby(_, :id) |> transform(_, :v=>mean)

,id,v,id2,v_mean
,Char,Float64,Int64,Float64
1,b,0.494644,1,0.489617
2,a,0.720003,2,0.540168
3,a,0.221093,3,0.540168
4,a,0.27621,4,0.540168
5,d,0.205227,5,0.529731
6,c,0.624687,6,0.50889
7,c,0.397678,7,0.50889
8,b,0.464126,8,0.489617
9,a,0.993339,9,0.540168


In [31]:
# note that combine reorders rows by group of GroupedDataFrame
@pipe x |> groupby(_, :id) |> combine(_, :id2, :v=>mean)

,id,id2,v_mean
,Char,Int64,Float64
1,b,1,0.489617
2,b,8,0.489617
3,b,14,0.489617
4,b,16,0.489617
5,b,19,0.489617
6,b,29,0.489617
7,b,39,0.489617
8,b,41,0.489617
9,b,42,0.489617


In [32]:
# we give a custom name for the result column
@pipe x |> groupby(_, :id) |> combine(_, :v=>mean=>:res)

,id,res
,Char,Float64
1,b,0.489617
2,a,0.540168
3,d,0.529731
4,c,0.50889


In [33]:
# you can have multiple operations
@pipe x |> groupby(_, :id) |> combine(_, :v=>mean=>:res1, :v=>sum=>:res2, nrow=>:n)

,id,res1,res2,n
,Char,Float64,Float64,Int64
1,b,0.489617,11.2612,23
2,a,0.540168,12.964,24
3,d,0.529731,11.6541,22
4,c,0.50889,15.7756,31


Additional notes:
* `select!` and `transform!` perform operations in-place
* The general syntax for transformation is `source_columns => function => target_column`
* if you pass multiple columns to a function they are treated as positional arguments
* `ByRow` and `AsTable` work exactly like discussed for operations on data frames in 05_columns.ipynb
* you can automatically groupby again the result of `combine`, `select` etc. by passing `ungroup=false` keyword argument to them
* similarly `keepkeys` keyword argument allows you to drop grouping columns from the resulting data frame

It is also allowed to pass a function to all these functions (also - as a special case, as a first argument). In this case the return value can be a table. In particular it allows for an easy dropping of groups if you return an empty table from the function.

If you pass a function you can use a `do` block syntax. In case of passing a function it gets a `SubDataFrame` as its argument.

Here is an example:

In [34]:
combine(groupby(x, :id)) do sdf
    n = nrow(sdf)
    n < 25 ? DataFrame() : DataFrame(n=n) # drop groups with low number of rows
end

,id,n
,Char,Int64
1,c,31


You can also produce multiple columns in a single operation, e.g.:

In [35]:
df = DataFrame(id=[1,1,2,2], val=[1,2,3,4])

,id,val
,Int64,Int64
1,1,1
2,1,2
3,2,3
4,2,4


In [36]:
@pipe df |> groupby(_, :id) |> combine(_, :val => (x -> [x]) => AsTable)

,id,x1,x2
,Int64,Int64,Int64
1,1,1,2
2,2,3,4


In [37]:
@pipe df |> groupby(_, :id) |> combine(_, :val => (x -> [x]) => [:c1, :c2])

,id,c1,c2
,Int64,Int64,Int64
1,1,1,2
2,2,3,4


### Aggregation of a data frame using `mapcols`

In [38]:
x = DataFrame(rand(10, 10), :auto)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0966498,0.759763,0.562443,0.38894,0.411924,0.538266,0.297264,0.711936
2,0.504155,0.111741,0.222208,0.512403,0.364941,0.806049,0.755719,0.630507
3,0.131687,0.349729,0.384267,0.0517748,0.124143,0.543742,0.197219,0.807013
4,0.624309,0.839581,0.555312,0.591734,0.178142,0.785604,0.0835652,0.570699
5,0.0522465,0.899302,0.534734,0.487683,0.443829,0.436251,0.388639,0.508346
6,0.847171,0.613108,0.883328,0.125407,0.433376,0.642765,0.710859,0.247029
7,0.323028,0.0143271,0.70193,0.908305,0.975363,0.863064,0.625343,0.668361
8,0.477463,0.254007,0.674406,0.63871,0.891207,0.307283,0.430385,0.843707
9,0.575513,0.312003,0.00964212,0.598696,0.790431,0.75908,0.939931,0.666364


In [39]:
mapcols(mean, x)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.369053,0.478217,0.518715,0.527864,0.487087,0.643584,0.530948,0.637022


### Mapping rows and columns using `eachcol` and `eachrow`

In [40]:
map(mean, eachcol(x)) # map a function over each column and return a vector

10-element Array{Float64,1}:
 0.3690529289039059
 0.47821651924120656
 0.5187154604905209
 0.5278640303308
 0.4870868703188103
 0.6435841691148785
 0.530948128024424
 0.6370223121108235
 0.4188901036310245
 0.4796283364142309

In [41]:
# an iteration returns a Pair with column name and values
foreach(c -> println(c[1], ": ", mean(c[2])), pairs(eachcol(x)))

x1: 0.3690529289039059
x2: 0.47821651924120656
x3: 0.5187154604905209
x4: 0.5278640303308
x5: 0.4870868703188103
x6: 0.6435841691148785
x7: 0.530948128024424
x8: 0.6370223121108235
x9: 0.4188901036310245
x10: 0.4796283364142309


In [42]:
# now the returned value is DataFrameRow which works as a NamedTuple but is a view to a parent DataFrame
map(r -> r.x1/r.x2, eachrow(x))

10-element Array{Float64,1}:
  0.1272105040056249
  4.511825317637913
  0.3765416376018312
  0.743596019284321
  0.0580967010964789
  1.3817632340717454
 22.546594714959323
  1.8797217080845827
  1.844574399600223
  0.0927546582403427

In [43]:
# it prints like a data frame, only the caption is different so that you know the type of the object
er = eachrow(x)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0966498,0.759763,0.562443,0.38894,0.411924,0.538266,0.297264,0.711936
2,0.504155,0.111741,0.222208,0.512403,0.364941,0.806049,0.755719,0.630507
3,0.131687,0.349729,0.384267,0.0517748,0.124143,0.543742,0.197219,0.807013
4,0.624309,0.839581,0.555312,0.591734,0.178142,0.785604,0.0835652,0.570699
5,0.0522465,0.899302,0.534734,0.487683,0.443829,0.436251,0.388639,0.508346
6,0.847171,0.613108,0.883328,0.125407,0.433376,0.642765,0.710859,0.247029
7,0.323028,0.0143271,0.70193,0.908305,0.975363,0.863064,0.625343,0.668361
8,0.477463,0.254007,0.674406,0.63871,0.891207,0.307283,0.430385,0.843707
9,0.575513,0.312003,0.00964212,0.598696,0.790431,0.75908,0.939931,0.666364


In [44]:
er.x1 # you can access columns of a parent data frame directly

10-element Array{Float64,1}:
 0.09664982449109094
 0.5041549474701863
 0.13168736619075805
 0.624309410304785
 0.05224650361175809
 0.8471705529645883
 0.3230280600962392
 0.4774632976198079
 0.575513492763198
 0.058305833526646644

In [45]:
# it prints like a data frame, only the caption is different so that you know the type of the object
ec = eachcol(x)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0966498,0.759763,0.562443,0.38894,0.411924,0.538266,0.297264,0.711936
2,0.504155,0.111741,0.222208,0.512403,0.364941,0.806049,0.755719,0.630507
3,0.131687,0.349729,0.384267,0.0517748,0.124143,0.543742,0.197219,0.807013
4,0.624309,0.839581,0.555312,0.591734,0.178142,0.785604,0.0835652,0.570699
5,0.0522465,0.899302,0.534734,0.487683,0.443829,0.436251,0.388639,0.508346
6,0.847171,0.613108,0.883328,0.125407,0.433376,0.642765,0.710859,0.247029
7,0.323028,0.0143271,0.70193,0.908305,0.975363,0.863064,0.625343,0.668361
8,0.477463,0.254007,0.674406,0.63871,0.891207,0.307283,0.430385,0.843707
9,0.575513,0.312003,0.00964212,0.598696,0.790431,0.75908,0.939931,0.666364


In [46]:
ec.x1 # you can access columns of a parent data frame directly

10-element Array{Float64,1}:
 0.09664982449109094
 0.5041549474701863
 0.13168736619075805
 0.624309410304785
 0.05224650361175809
 0.8471705529645883
 0.3230280600962392
 0.4774632976198079
 0.575513492763198
 0.058305833526646644

### Transposing

you can transpose a data frame using `permutedims`:

In [47]:
df = DataFrame(reshape(1:12, 3, 4), :auto)

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,4,7,10
2,2,5,8,11
3,3,6,9,12


In [48]:
df.names = ["a", "b", "c"]

3-element Array{String,1}:
 "a"
 "b"
 "c"

In [49]:
permutedims(df, :names)

,names,a,b,c
,String,Int64,Int64,Int64
1,x1,1,2,3
2,x2,4,5,6
3,x3,7,8,9
4,x4,10,11,12
